### 寻找排序数组中的最小值2

![微信截图_20190913130526.png](https://i.loli.net/2019/09/14/Jw8STysUZ3xpWP2.png)

- 这道题关键在于m和r的对比判断
- 由于存在重复，当m和r的值相同时，没法判断最小值向哪里进行二分
- 所以直接令 r -= 1,这样就可以了

In [ ]:
class Solution(object):
    def findMin(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        l,r = 0, len(nums)-1
        while l < r:
            m = (l+r)//2
            if nums[m] < nums[r]:
                r = m
            elif nums[m] > nums[r]:
                l = m + 1
            else:
                r -= 1
        return nums[l]

### 寻找重复数

![微信截图_20190914151724.png](https://i.loli.net/2019/09/15/4fCNaW1H26mYqdw.png)

In [4]:
class Solution(object):
    def findDuplicate(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        l, r = 0, len(nums)-1
        while l < r:
            m = (l+r)//2
            c = 0
            for n in nums:
                if n <= m:
                    c += 1
            if c > m:
                r = m
            else:
                l = m + 1
        return l

### 寻找两个有序数组的中位数

![微信截图_20190914154217.png](https://i.loli.net/2019/09/15/M2ExNrSzKkRLf3P.png)

- 自己想的双指针法
- 不符合时间复杂度$O(log(m+n))$的规定

In [5]:
class Solution(object):
    def findMedianSortedArrays(self, nums1, nums2):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :rtype: float
        """
        n, m = len(nums1), len(nums2)
        res, i, j = [], 0, 0
        
        while i < len(nums1) and j < len(nums2):
            if nums1[i] <= nums2[j]:
                res.append(nums1[i])
                i += 1
            else:
                res.append(nums2[j])
                j += 1
                
        if i == n: res += nums2[j:]
        if j == m: res += nums1[i:]
        
        if (n+m)%2 == 0:
            return (res[(n+m)//2]+res[((n+m)//2)-1])/float(2)
        else:
            return res[(n+m)//2]

- 非常复杂的$O(log(m+n))$解法
- [参考](https://leetcode-cn.com/problems/median-of-two-sorted-arrays/solution/he-bing-yi-hou-zhao-gui-bing-guo-cheng-zhong-zhao-/)

In [12]:
class Solution(object):
    def findMedianSortedArrays(self, nums1, nums2):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :rtype: float
        """
        if len(nums1) > len(nums2):
            nums1, nums2 = nums2, nums1
            
        m, n  = len(nums1), len(nums2)
        l, r = 0, m
        left_total = (m+n+1)//2
        
        while l < r:
            i = (l+r)//2
            j = left_total-i
            
            if nums2[j-1] > nums1[i]:
                l = i + 1
            else:
                r = i
        
        i, j = l, left_total-l
        
        nums1_left_max = float('-inf') if i==0 else nums1[i-1]
        nums1_right_min = float('inf') if i==m else nums1[i]
        
        nums2_left_max = float('-inf') if j==0 else nums2[j-1]
        nums2_right_min = float('inf') if j==n else nums2[j]
        
        if (m+n) & 1:
            return max(nums1_left_max,nums2_left_max)
        else:
            return (max(nums1_left_max,nums2_left_max)+min(nums1_right_min,nums2_right_min))/2

In [16]:
a = Solution()
x = [1,2]
y = [3,4]
a.findMedianSortedArrays(x,y)

2.5

### 找出第K小的距离对

![微信截图_20190915143648.png](https://i.loli.net/2019/09/16/wUAM5tCN6RHOElZ.png)

- 实际上，题目要求我们寻找的是第K小的距离！我们知道，距离是由数组中的一个数和另一个数相减再取绝对值得到。那么我们可以这样操作：把数组排好序，然后用最后一个数减去第一个数，得到的就是这个数组的最大距离！显然，第K小的距离，一定会落在距离最小值和距离最大值之间,即\[0, nums[-1] - nums[0]\]这个空间，最小值不一定是0，但我们可以0作为这个假设空间的下限，然后通过二分法去逼近这个第K小的距离。<br><br>
- 怎么找这个第K小的距离呢？首先我们猜测一个距离guess，然后计算一下，数组中有多少个距离，小于等于guess这个距离。假设我们计算得到count，则说明数组有count对数的距离，都是小于等于guess的。接着，再把这个count跟k做对比，如果count是大于等于k的，说明我们猜的距离大了，应该猜小点，如果count是小于k的，说明我们猜小了，应该猜大点。你可能会问，有没有可能猜的刚好等于k呢？实际上是有可能的！但是我们需要借助二分法不断去逼近正确的距离，而不是试图一步到位。

- 如何计算数组中有多少个距离，小于等于猜测距离呢？可以通过双指针法来维护一个窗口。在这个窗口中，如果右指针对应的数，减去左指针对应的数，其值(即距离)，小于等于我们猜测的距离，则说明目前有右指针减去左指针这么多的距离都是符合要求的，所以我们把它累加起来，如果计算距离大于猜测距离，左指针累加1即可，直到计算距离再度小于等于猜测距离。

In [12]:
class Solution(object):
    def smallestDistancePair(self, nums, k):
        def possible(guess):
            count = left = 0
            for right, x in enumerate(nums):
                while x - nums[left] > guess:
                    left += 1
                count += right - left
            return count >= k

        nums.sort()
        lo, hi = 0, nums[-1] - nums[0]
        while lo < hi:
            mi = (lo + hi) / 2
            if possible(mi):
                hi = mi
            else:
                lo = mi + 1
        return lo

In [13]:
nums = [1,3,1]
k = 1
a = Solution()

In [14]:
a.smallestDistancePair(nums,k)

0

### 分割数组的最大值

![微信截图_20190917141952.png](https://i.loli.net/2019/09/18/TKSzXw9hUeQtsd8.png)

- 首先要理解题目的意思！题目的意思是说，要把数组分成m个子数组，然后这m个子数组分别求和得到m个和，再求出这m个和的最大值max。我们知道，把数组分成m个子数组有很多种不同的分法，不同分法下，对应的子数组之和是不一样的，所以得到的最大值max也是不一样的，我们的目标就是要找到这些不一样的最大值中最小的那一个最大值。<br><br>
- 首先来分析，如果m和数组 nums 中元素的个数相等，那么每个元素都是一个子数组，所以返回 nums 中最大的数字即可。如果m为1，那么整个 nums 数组就是一个子数组，返回 nums 所有数字之和，所以对于其他有效的m值，返回的值必定在上面两个值之间，只要我们设定好条件，就可以用二分搜索法来找到最优的那个最大值。<br><br>
- 那么条件如何设定呢？我们取区间的中值mid，然后在nums中进行元素累加，每次累加一旦大于mid，就记为增加一组！注意，组数要从1开始。如果最后算出来的组数是比m大的，说明mid小了。反之则是mid大了。<br><br>
- 用一个例子来分析，nums = [1, 2, 3, 4, 5], m = 3，将 left 设为数组中的最大值5，right 设为数字之和 15，然后算出中间数为 10，接下来要做的是找出和最大且小于等于 10 的子数组的个数，[1, 2, 3, 4], [5]，可以看到无法分为3组，说明 mid 偏大，所以让 right=mid，然后再次进行二分查找，算出 mid=7，再次找出和最大且小于等于7的子数组的个数，[1,2,3], [4], [5]，成功的找出了三组，说明 mid 还可以进一步降低，让 right=mid，再次进行二分查找，算出 mid=6，再次找出和最大且小于等于6的子数组的个数，[1,2,3], [4], [5]，成功的找出了三组，尝试着继续降低 mid，让 right=mid，再次进行二分查找，算出 mid=5，再次找出和最大且小于等于5的子数组的个数，[1,2], [3], [4], [5]，发现有4组，此时的 mid 太小了，应该增大 mid，让 left=mid+1，此时 left=6，right=5，循环退出了，返回 left 即可

In [1]:
class Solution(object):
    def splitArray(self, nums, m):
        """
        :type nums: List[int]
        :type m: int
        :rtype: int
        """
        def f(mid):
            temp, count = 0, 1
            for n in nums:
                temp += n
                if temp > mid:
                    count += 1
                    temp = n
            return count
        
        l, r = max(nums), sum(nums)
        
        while l < r:
            mid = (l+r)//2
            if f(mid) > m:
                l = mid + 1
            else:
                r = mid
                
        return l